### Objective

In this notebook, we generate samples for the input paramters for thermal analysis. Here, we condsider two heat sources.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import qmc
from sklearn.cluster import KMeans
from collections import defaultdict

In [2]:
# Define parameters
Ta = 25
c_source = 50e-3
d_source = 65e-3
c_module = 61.4e-3
d_module = 106e-3
fan = 3

# Generate samples
data_length = 50000
data_number = 8
append_position_number = 1
sampler = qmc.LatinHypercube(d=data_number)
X = sampler.random(n=data_length)

In [3]:
# Scale X to the required ranges
Q1_min, Q1_max = 50, 400
Q1_sum = (Q1_max - Q1_min) * X[:, 0] + Q1_min

Q2_min, Q2_max = 50, 400
Q2_sum = (Q2_max - Q2_min) * X[:, 1] + Q2_min

d_min, d_max = 5e-3, 30e-3
d_sum = (d_max - d_min) * X[:, 2] + d_min

b_min, b_max = 73.7e-3, 307e-3
b_sum = (b_max - b_min) * X[:, 3] + b_min

L_min, L_max = 127.2e-3, 530e-3
L_sum = (L_max - L_min) * X[:, 4] + L_min

c_min, c_max = 10e-3, 39e-3
c_sum = (c_max - c_min) * X[:, 5] + c_min

L_duct_min, L_duct_max = 20e-3, 50e-3
L_duct_sum = (L_duct_max - L_duct_min) * X[:, 6] + L_duct_min

n_min, n_max = 10, 50
n_sum = np.round((n_max - n_min) * X[:, 7] + n_min).astype(int)

data_sum = np.column_stack((Q1_sum, Q2_sum, d_sum, b_sum, L_sum, c_sum, L_duct_sum, n_sum))

In [4]:
result_design = []
t_invalid = 0
valid_position = defaultdict(list)

# Derived parameters calculation
for i in range(data_length):
    if (i+1)%500 == 0:
        print(f"Processing {i+1} samples")
        
    Q1, Q2, d, b, L, c, L_duct, n = data_sum[i]

    t_min = 1e-3
    t_max = b / n - 1e-3

    if t_min > t_max:
        t_invalid += 1
    
    else:
        t = np.random.rand() * (t_max - t_min) + t_min

        Xc_min = c_module / 2
        Xc_max = b - c_module / 2
        Yc_min = d_module / 2
        Yc_max = L - d_module / 2

        # Generate position samples 
        position_sampler = qmc.LatinHypercube(d=4)
        positions = position_sampler.random(n=int(data_length/10))

        # Scale samples
        positions[:, 0] = positions[:, 0]*(Xc_max-Xc_min)+ Xc_min;
        positions[:, 1] = positions[:, 1]*(Yc_max-Yc_min)+ Yc_min;
        positions[:, 2] = positions[:, 2]*(Xc_max-Xc_min)+ Xc_min;
        positions[:, 3] = positions[:, 3]*(Yc_max-Yc_min)+ Yc_min;

        # Check non-overlapping
        xc1, yc1, xc2, yc2 = positions[:, 0], positions[:, 1], positions[:, 2], positions[:, 3]
        non_overlapping = (np.abs(xc1 - xc2) > c_module) | (np.abs(yc1 - yc2) > d_module)

        # Retain valid positions
        valid_positions = positions[non_overlapping]
        valid_position['b'].append(b)
        valid_position['L'].append(L)
        valid_position['valid_pos'].append(len(valid_positions))

        # Compose the design variables
        if len(valid_positions) > 0:
            
            if append_position_number == 1:
                # Randomly pick 1 sample from the valid positions
                random_index = np.random.randint(0, len(valid_positions))
                selected_position = valid_positions[random_index]
                result_design.append(np.concatenate((data_sum[i], [t], selected_position)))
                
            elif len(valid_positions) <= append_position_number:
                for pos in valid_positions:
                    # Append each valid position to the design
                    result_design.append(np.concatenate((data_sum[i], [t], pos)))
    
            else:
                # Perform clustering and append the centroid/closest to centroid positions
                kmeans = KMeans(n_clusters=append_position_number, n_init=10, random_state=0).fit(valid_positions)
                centers = kmeans.cluster_centers_
                
                # For each center, find the closest valid position
                for center in centers:
                    distances = np.sqrt(((valid_positions - center)**2).sum(axis=1))
                    closest_index = np.argmin(distances)
                    closest_position = valid_positions[closest_index]
                    
                    # Append the closest valid position to the design
                    result_design.append(np.concatenate((data_sum[i], [t], closest_position)))

# Swap Q1 & Q2 values to ensure Q1 always larger than Q2
result_design = np.array(result_design)
swap = result_design[:, 0] < result_design[:, 1]
result_design[swap, 0], result_design[swap, 1] = result_design[swap, 1], result_design[swap, 0].copy()

# Swap locations as well
result_design[swap, -4], result_design[swap, -2] = result_design[swap, -2], result_design[swap, -4].copy()
result_design[swap, -3], result_design[swap, -1] = result_design[swap, -1], result_design[swap, -3].copy()

Processing 500 samples
Processing 1000 samples
Processing 1500 samples
Processing 2000 samples
Processing 2500 samples
Processing 3000 samples
Processing 3500 samples
Processing 4000 samples
Processing 4500 samples
Processing 5000 samples
Processing 5500 samples
Processing 6000 samples
Processing 6500 samples
Processing 7000 samples
Processing 7500 samples
Processing 8000 samples
Processing 8500 samples
Processing 9000 samples
Processing 9500 samples
Processing 10000 samples
Processing 10500 samples
Processing 11000 samples
Processing 11500 samples
Processing 12000 samples
Processing 12500 samples
Processing 13000 samples
Processing 13500 samples
Processing 14000 samples
Processing 14500 samples
Processing 15000 samples
Processing 15500 samples
Processing 16000 samples
Processing 16500 samples
Processing 17000 samples
Processing 17500 samples
Processing 18000 samples
Processing 18500 samples
Processing 19000 samples
Processing 19500 samples
Processing 20000 samples
Processing 20500 sam

In [5]:
df = pd.DataFrame(valid_position)

In [6]:
print(f"Invalid t design: {t_invalid}/{data_length}")
print(f"Invalid heat source position: {np.sum(df['valid_pos'] == 0)}/{len(df)}")
print(f"Invalid heat source position ratio (%): {np.sum(df['valid_pos'] == 0)/len(df)*100:.3f}%")
print(f"Remaining valid samples: {result_design.shape[0]}/{data_length}")

Invalid t design: 924/50000
Invalid heat source position: 2091/49076
Invalid heat source position ratio (%): 4.261%
Remaining valid samples: 46985/50000


In [7]:
# Configure dataframe
design_df = pd.DataFrame(result_design)
column_names = ['Q1', 'Q2', 'd', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2']
design_df.columns = column_names
design_df

,Q1,Q2,d,b,L,c,L_duct,n,t,xc1,yc1,xc2,yc2
0,240.052301,131.206556,0.022801,0.241861,0.309655,0.024212,0.021753,28.0,0.007413,0.192554,0.215502,0.059563,0.070609
1,176.536197,108.320199,0.017852,0.181335,0.305039,0.029720,0.022363,37.0,0.003675,0.099735,0.095076,0.116866,0.215334
2,211.790291,194.688015,0.022246,0.279266,0.215025,0.019742,0.023158,22.0,0.010453,0.186667,0.053082,0.072310,0.110853
3,311.915342,127.148073,0.007785,0.207182,0.300995,0.015386,0.035060,44.0,0.002303,0.082717,0.229144,0.155975,0.100768
4,207.663582,160.755036,0.019127,0.274774,0.328928,0.012300,0.042063,37.0,0.002363,0.240603,0.064858,0.153940,0.166586
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46980,196.321154,127.663371,0.024598,0.282231,0.142000,0.021599,0.030333,39.0,0.004817,0.215480,0.085525,0.118855,0.073038
46981,352.003274,127.159545,0.012094,0.109613,0.456309,0.035968,0.045920,40.0,0.001046,0.050375,0.363596,0.048657,0.256411
46982,335.680712,77.071109,0.023446,0.193423,0.301259,0.014391,0.039784,45.0,0.003141,0.138793,0.186242,0.030989,0.137444
46983,351.576806,306.925684,0.022301,0.212533,0.419421,0.019623,0.040883,12.0,0.009266,0.034857,0.292895,0.114214,0.239866


In [8]:
# Save the design samples
design_df.to_csv("AL_pool.csv", index=False)